In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 157]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
13150  prop-1      157    9    5    0   27   45    32   5  23  ...  0.0    0   
13151  prop-1      157    3    4    0    1    4     3   0   1  ...  0.0    0   
13152  prop-1      157   12    2    0   76   24    54  71   5  ...  1.0    0   
13153  prop-1      157    2    1    0    3    2     1   3   0  ...  0.0    0   
13154  prop-1      157    2    1    0   12    2     1  12   0  ...  0.0    0   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
15641  prop-1      157    3    3    0    8   11     1   0   8  ...  1.0    0   
15642  prop-1      157    4    2    0    5    6     6   0   5  ...  0.0    0   
15643  prop-1      157    8    5    0   15   12    22  13   2  ...  1.0    0   
15644  prop-1      157    1    1    0    4    1     0   2   2  ...  0.0    0   
15645  prop-1      157    8    1    0   25   44    16   9  16  ...  1.0    0   

            mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
13150  0.931034  0.333333   2    7  18.777778       5  1.3333    4  
13151  0.800000  1.000000   1    1   2.333333       1  0.6667    0  
13152  0.500000  0.226190   1    1   7.833333       1  0.9167    0  
13153  0.000000  1.000000   0    0   0.000000       1  1.0000    0  
13154  0.000000  1.000000   0    0   0.000000       1  1.0000    0  
...         ...       ...  ..  ...        ...     ...     ...  ...  
15641  0.947368  0.666667   0    0   9.000000       2  0.6667    0  
15642  0.571429  0.562500   1    1   2.750000       1  0.7500    0  
15643  0.928571  0.575000   1    1   5.250000       1  0.2500    0  
15644  0.000000  1.000000   0    0   0.000000       1  1.0000    0  
15645  0.000000  0.457143   0    0  39.250000       2  1.1250    0  

[2496 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
14086    3    1    0    4    8     3   1   3    2  2.000000   40  0.0    0   
14079    2    1    0    7   11     1   2   5    2  2.000000   65  0.0    0   
14060   11    5    0   14   52     7   8  12    4  0.812500  305  1.0    2   
15482    2    1    0    8    2     1   5   3    2  2.000000    2  0.0    0   
13894    8    3    0   27   67    26   0  27    2  0.857143  726  1.0    0   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
13924    4    5    0    2    6     0   0   2    4  0.666667   39  1.0    0   
14630    7    1    0   16   30     9   4  12    3  0.583333  130  1.0    1   
14436    3    3    0   16   19     3   0  16    2  2.000000   77  0.0    0   
13313    0    1    0    0    0     0   0   0    0  2.000000    0  0.0    0   
15131    2    3    0    8   13     1   1   7    1  2.000000   39  0.0    0   

            mfa       cam  ic  cbm        amc  max_cc  avg_cc  
14086  0.000000  0.333333   0    0  12.333333       1  0.6667  
14079  0.000000  0.750000   0    0  31.500000       3  1.5000  
14060  0.983306  0.272727   1    1  26.000000       4  1.4545  
15482  0.000000  0.833333   0    0   0.000000       1  1.0000  
13894  0.500000  0.380952   1    1  89.625000      28  7.1250  
...         ...       ...  ..  ...        ...     ...     ...  
13924  0.962963  0.625000   1    1   8.250000       1  0.2500  
14630  0.000000  0.291667   0    0  17.285714       5  1.5714  
14436  0.750000  0.555556   0    0  24.666667       7  3.0000  
13313  0.000000  0.000000   0    0   0.000000       0  0.0000  
15131  0.941176  0.625000   1    2  18.500000       1  0.5000  

[1996 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
15466    4    6    0   30    8     6  29   1    4  1.333333   25  1.0    0   
15619    9    2    0   25   54    14   0  25    2  0.775000  428  1.0    0   
14814    3    2    0    4    5     3   1   3    2  1.000000   17  1.0    0   
15019    1    1    0    3    1     0   1   2    1  2.000000    1  0.0    0   
14653    1    1    0    5    1     0   3   2    1  2.000000    1  0.0    0   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
14188    2    1    0    9   17     1   0   9    2  2.000000   44  0.0    0   
14312    4    2    0   12   39     0   0  12    2  2.000000  164  0.0    0   
13856    2    1    0    4    3     1   0   4    2  2.000000    6  0.0    0   
13721    5    4    1    1    9     4   1   0    5  0.875000   32  1.0    0   
13984    5    2    4   18   24    10   4  14    2  2.000000  135  0.0    0   

            mfa       cam  ic  cbm        amc  max_cc  avg_cc  
15466  1.000000  0.750000   0    0   5.000000       0  0.0000  
15619  0.666667  0.375000   0    0  46.000000       9  2.5556  
14814  0.923077  0.625000   1    1   4.333333       1  0.3333  
15019  0.000000  1.000000   0    0   0.000000       1  1.0000  
14653  0.000000  1.000000   0    0   0.000000       1  1.0000  
...         ...       ...  ..  ...        ...     ...     ...  
14188  0.000000  0.666667   0    0  21.000000       1  0.5000  
14312  0.833333  0.375000   0    0  40.000000       3  1.2500  
13856  0.000000  0.600000   0    0   2.000000       1  0.5000  
13721  0.944444  0.600000   0    0   5.000000       1  0.2000  
13984  0.500000  0.288889   1    1  26.000000       1  0.8000  

[500 rows x 20 columns]

In [9]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [10]:
y_train

bug
14086    0
14079    0
14060    0
15482    0
13894    0
...    ...
13924    0
14630    0
14436    1
13313    0
15131    0

[1996 rows x 1 columns]

In [11]:
y_test

bug
15466    0
15619    0
14814    0
15019    0
14653    0
...    ...
14188    0
14312    0
13856    0
13721    0
13984    0

[500 rows x 1 columns]

In [12]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [13]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [14]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [15]:
y = y_test['bug'].tolist()
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 7,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 3,
 0,
 7,
 1,
 0,
 0,
 4,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,

In [16]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv157/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
20/20 [==============================] - 0s 2ms/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 4/100
20/20 [==============================] - 0s 912us/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 

20/20 [==============================] - 0s 952us/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 89/100
20/20 [==============================] - 0s 1ms/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 90/100
20/20 [==============================] - 0s 957us/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 91/100
20/20 [==============================] - 0s 989us/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 92/100
20/20 [==============================] - 0s 2ms/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 93/100
20/20 [==============================] - 0s 1ms/step - loss: 0.8808 - mse:

20/20 [==============================] - 0s 870us/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 74/100
20/20 [==============================] - 0s 827us/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 75/100
20/20 [==============================] - 0s 859us/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 76/100
20/20 [==============================] - 0s 826us/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 77/100
20/20 [==============================] - 0s 855us/step - loss: 0.8808 - mse: 0.8808 - mae: 0.2886 - root_mean_squared_error: 0.9385 - mean_squared_logarithmic_error: 0.1686
Epoch 78/100
20/20 [==============================] - 0s 806us/step - loss: 0.8808 

20/20 [==============================] - 0s 871us/step - loss: 30.7141 - mse: 30.7141 - mae: 0.6214 - root_mean_squared_error: 5.5420 - mean_squared_logarithmic_error: 0.2358
Epoch 58/100
20/20 [==============================] - 0s 829us/step - loss: 29.1896 - mse: 29.1896 - mae: 0.6098 - root_mean_squared_error: 5.4027 - mean_squared_logarithmic_error: 0.2340
Epoch 59/100
20/20 [==============================] - 0s 818us/step - loss: 27.0619 - mse: 27.0619 - mae: 0.5945 - root_mean_squared_error: 5.2021 - mean_squared_logarithmic_error: 0.2317
Epoch 60/100
20/20 [==============================] - 0s 816us/step - loss: 25.4796 - mse: 25.4796 - mae: 0.5810 - root_mean_squared_error: 5.0477 - mean_squared_logarithmic_error: 0.2296
Epoch 61/100
20/20 [==============================] - 0s 1ms/step - loss: 23.9549 - mse: 23.9549 - mae: 0.5688 - root_mean_squared_error: 4.8944 - mean_squared_logarithmic_error: 0.2279
Epoch 62/100
20/20 [==============================] - 0s 1ms/step - loss: 2

20/20 [==============================] - 0s 910us/step - loss: 109.0946 - mse: 109.0946 - mae: 0.6276 - root_mean_squared_error: 10.4448 - mean_squared_logarithmic_error: 0.2164
Epoch 41/100
20/20 [==============================] - 0s 826us/step - loss: 106.3590 - mse: 106.3590 - mae: 0.6221 - root_mean_squared_error: 10.3131 - mean_squared_logarithmic_error: 0.2158
Epoch 42/100
20/20 [==============================] - 0s 774us/step - loss: 104.0112 - mse: 104.0112 - mae: 0.6143 - root_mean_squared_error: 10.1986 - mean_squared_logarithmic_error: 0.2146
Epoch 43/100
20/20 [==============================] - 0s 799us/step - loss: 101.4415 - mse: 101.4415 - mae: 0.6100 - root_mean_squared_error: 10.0718 - mean_squared_logarithmic_error: 0.2141
Epoch 44/100
20/20 [==============================] - 0s 783us/step - loss: 100.4759 - mse: 100.4759 - mae: 0.6049 - root_mean_squared_error: 10.0238 - mean_squared_logarithmic_error: 0.2132
Epoch 45/100
20/20 [==============================] - 0s 8

...

20/20 [==============================] - 0s 1ms/step - loss: 0.7589 - mse: 0.7589 - mae: 0.3868 - root_mean_squared_error: 0.8712 - mean_squared_logarithmic_error: 0.1464
Epoch 96/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7600 - mse: 0.7600 - mae: 0.3866 - root_mean_squared_error: 0.8718 - mean_squared_logarithmic_error: 0.1469
Epoch 97/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7625 - mse: 0.7625 - mae: 0.3829 - root_mean_squared_error: 0.8732 - mean_squared_logarithmic_error: 0.1460
Epoch 98/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7640 - mse: 0.7640 - mae: 0.4022 - root_mean_squared_error: 0.8741 - mean_squared_logarithmic_error: 0.1519
Epoch 99/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7668 - mse: 0.7668 - mae: 0.3892 - root_mean_squared_error: 0.8757 - mean_squared_logarithmic_error: 0.1481
Epoch 100/100
16/16 [==============================] - 0s 807us/step - loss: 1.0017 - mse: 1.

20/20 [==============================] - 0s 1ms/step - loss: 0.8742 - mse: 0.8742 - mae: 0.2991 - root_mean_squared_error: 0.9350 - mean_squared_logarithmic_error: 0.1671
Epoch 82/100
20/20 [==============================] - 0s 1ms/step - loss: 0.8742 - mse: 0.8742 - mae: 0.2995 - root_mean_squared_error: 0.9350 - mean_squared_logarithmic_error: 0.1672
Epoch 83/100
20/20 [==============================] - 0s 1ms/step - loss: 0.8742 - mse: 0.8742 - mae: 0.3008 - root_mean_squared_error: 0.9350 - mean_squared_logarithmic_error: 0.1673
Epoch 84/100
20/20 [==============================] - 0s 1ms/step - loss: 0.8740 - mse: 0.8740 - mae: 0.3010 - root_mean_squared_error: 0.9349 - mean_squared_logarithmic_error: 0.1672
Epoch 85/100
20/20 [==============================] - 0s 982us/step - loss: 0.8741 - mse: 0.8741 - mae: 0.3002 - root_mean_squared_error: 0.9349 - mean_squared_logarithmic_error: 0.1672
Epoch 86/100
20/20 [==============================] - 0s 1ms/step - loss: 0.8740 - mse: 0.8

20/20 [==============================] - 0s 1ms/step - loss: 77.0423 - mse: 77.0423 - mae: 0.9662 - root_mean_squared_error: 8.7774 - mean_squared_logarithmic_error: 0.2987
Epoch 66/100
20/20 [==============================] - 0s 1ms/step - loss: 74.2515 - mse: 74.2515 - mae: 0.9464 - root_mean_squared_error: 8.6169 - mean_squared_logarithmic_error: 0.2956
Epoch 67/100
20/20 [==============================] - 0s 1ms/step - loss: 72.0106 - mse: 72.0106 - mae: 0.9259 - root_mean_squared_error: 8.4859 - mean_squared_logarithmic_error: 0.2911
Epoch 68/100
20/20 [==============================] - 0s 1ms/step - loss: 68.9166 - mse: 68.9166 - mae: 0.9034 - root_mean_squared_error: 8.3016 - mean_squared_logarithmic_error: 0.2872
Epoch 69/100
20/20 [==============================] - 0s 1ms/step - loss: 66.5835 - mse: 66.5835 - mae: 0.8867 - root_mean_squared_error: 8.1599 - mean_squared_logarithmic_error: 0.2840
Epoch 70/100
20/20 [==============================] - 0s 1ms/step - loss: 64.3158 -

20/20 [==============================] - 0s 1ms/step - loss: 8.3065 - mse: 8.3065 - mae: 0.5763 - root_mean_squared_error: 2.8821 - mean_squared_logarithmic_error: 0.2700
Epoch 50/100
20/20 [==============================] - 0s 1ms/step - loss: 7.6924 - mse: 7.6924 - mae: 0.5639 - root_mean_squared_error: 2.7735 - mean_squared_logarithmic_error: 0.2671
Epoch 51/100
20/20 [==============================] - 0s 1ms/step - loss: 7.3421 - mse: 7.3421 - mae: 0.5517 - root_mean_squared_error: 2.7096 - mean_squared_logarithmic_error: 0.2631
Epoch 52/100
20/20 [==============================] - 0s 1ms/step - loss: 6.7227 - mse: 6.7227 - mae: 0.5375 - root_mean_squared_error: 2.5928 - mean_squared_logarithmic_error: 0.2592
Epoch 53/100
20/20 [==============================] - 0s 1ms/step - loss: 6.3353 - mse: 6.3353 - mae: 0.5275 - root_mean_squared_error: 2.5170 - mean_squared_logarithmic_error: 0.2560
Epoch 54/100
20/20 [==============================] - 0s 1ms/step - loss: 6.0180 - mse: 6.018

20/20 [==============================] - 0s 1ms/step - loss: 1.1952 - mse: 1.1952 - mae: 0.3011 - root_mean_squared_error: 1.0932 - mean_squared_logarithmic_error: 0.1739
Epoch 35/100
20/20 [==============================] - 0s 1ms/step - loss: 1.0691 - mse: 1.0691 - mae: 0.2983 - root_mean_squared_error: 1.0340 - mean_squared_logarithmic_error: 0.1732
Epoch 36/100
20/20 [==============================] - 0s 1ms/step - loss: 1.0245 - mse: 1.0245 - mae: 0.2971 - root_mean_squared_error: 1.0122 - mean_squared_logarithmic_error: 0.1728
Epoch 37/100
20/20 [==============================] - 0s 1ms/step - loss: 0.9878 - mse: 0.9878 - mae: 0.2959 - root_mean_squared_error: 0.9939 - mean_squared_logarithmic_error: 0.1724
Epoch 38/100
20/20 [==============================] - 0s 1ms/step - loss: 0.9754 - mse: 0.9754 - mae: 0.2955 - root_mean_squared_error: 0.9876 - mean_squared_logarithmic_error: 0.1723
Epoch 39/100
20/20 [==============================] - 0s 1ms/step - loss: 0.9387 - mse: 0.938

In [17]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv157/'

In [18]:
total_training_time

[3.00075626373291,
 2.6431727409362793,
 2.594513177871704,
 2.4614360332489014,
 2.4382870197296143,
 2.4582014083862305,
 2.4838333129882812,
 2.538689613342285,
 2.780714750289917,
 2.81007719039917,
 2.8895695209503174,
 2.4531333446502686,
 2.498018503189087,
 2.4799153804779053,
 2.506237030029297,
 2.556549310684204,
 2.5778870582580566,
 2.4870822429656982,
 2.602569103240967,
 2.6412782669067383,
 2.489259958267212,
 2.520522356033325,
 2.5803146362304688,
 2.713663339614868,
 2.979149103164673,
 3.073673725128174,
 3.1113641262054443,
 3.0486812591552734,
 3.048111915588379,
 3.128146171569824]

In [19]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 2.6864935954411826


In [20]:
total_training_loss_history

[{'loss': [0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   0.8807615041732788,
   

In [21]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 3.390865981578827
avg mae of training: 0.33731615742047627


In [22]:
total_scores

[[1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [2.149050235748291,
  2.149050235748291,
  0.33859944343566895,
  1.4659639596939087,
  0.18349649012088776],
 [1.4111218452453613,
  1.4111218452453613,
  0.3134998381137848,
  1.1879065036773682,
  0.179740309715271],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [1.4029748439788818,
  1.4029748439788818,
  0.33035361766815186,
  1.1844724416732788,
  0.18635959923267365],
 [1.448383092880249,
  1.448383092880249,
  0.3200557827949524,
  1.2034878730773926,
  0.18355585634708405],
 [1.1871837377548218,
  1.1871837377548218,
  0.295790433883667,
  1.0895795822143555,
  0.16985942423343658

In [4]:
a = [[1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [2.149050235748291,
  2.149050235748291,
  0.33859944343566895,
  1.4659639596939087,
  0.18349649012088776],
 [1.4111218452453613,
  1.4111218452453613,
  0.3134998381137848,
  1.1879065036773682,
  0.179740309715271],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [1.4029748439788818,
  1.4029748439788818,
  0.33035361766815186,
  1.1844724416732788,
  0.18635959923267365],
 [1.448383092880249,
  1.448383092880249,
  0.3200557827949524,
  1.2034878730773926,
  0.18355585634708405],
 [1.1871837377548218,
  1.1871837377548218,
  0.295790433883667,
  1.0895795822143555,
  0.16985942423343658],
 [30.705875396728516,
  30.705875396728516,
  1.1627742052078247,
  5.541288375854492,
  0.4234684705734253],
 [1.2382992506027222,
  1.2382992506027222,
  0.303217351436615,
  1.1127889156341553,
  0.17450524866580963],
 [1.4022570848464966,
  1.4022570848464966,
  0.32929855585098267,
  1.1841694116592407,
  0.18904747068881989],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [1.002803087234497,
  1.002803087234497,
  0.2843129634857178,
  1.0014005899429321,
  0.15618516504764557],
 [1.1799999475479126,
  1.1799999475479126,
  0.2920001447200775,
  1.0862780809402466,
  0.16759923100471497],
 [1.1833056211471558,
  1.1833056211471558,
  0.2981529235839844,
  1.0877984762191772,
  0.16899576783180237],
 [21.800321578979492,
  21.800321578979492,
  0.47583895921707153,
  4.669081211090088,
  0.16591870784759521],
 [0.9291585683822632,
  0.9291585683822632,
  0.28499653935432434,
  0.9639286994934082,
  0.1585671752691269],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497],
 [1.2707082033157349,
  1.2707082033157349,
  0.3054690957069397,
  1.127256989479065,
  0.1759689748287201],
 [1.1725386381149292,
  1.1725386381149292,
  0.299551397562027,
  1.0828381776809692,
  0.15918035805225372],
 [1.2877062559127808,
  1.2877062559127808,
  0.3173198103904724,
  1.134771466255188,
  0.18092960119247437],
 [1.1937106847763062,
  1.1937106847763062,
  0.3298851251602173,
  1.0925706624984741,
  0.17257164418697357],
 [1.0017021894454956,
  1.0017021894454956,
  0.397148996591568,
  1.0008506774902344,
  0.1434386819601059],
 [1.1480447053909302,
  1.1480447053909302,
  0.3080402612686157,
  1.071468472480774,
  0.16489948332309723],
 [2.571260929107666,
  2.571260929107666,
  0.36344778537750244,
  1.6035151481628418,
  0.18624304234981537],
 [2.3949520587921143,
  2.3949520587921143,
  0.37432265281677246,
  1.5475633144378662,
  0.19537855684757233],
 [1.1799999475479126,
  1.1799999475479126,
  0.2919999957084656,
  1.0862780809402466,
  0.16759923100471497]]

In [5]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])
sum(test_mae)/10

0.3929073303937912

In [3]:
test_mae

[0.2919999957084656,
 0.2919999957084656,
 0.33859944343566895,
 0.3134998381137848,
 0.2919999957084656,
 0.2919999957084656,
 0.33035361766815186,
 0.3200557827949524,
 0.295790433883667,
 1.1627742052078247,
 0.303217351436615,
 0.32929855585098267,
 0.2919999957084656,
 0.2919999957084656,
 0.2919999957084656,
 0.2843129634857178,
 0.2920001447200775,
 0.2981529235839844,
 0.47583895921707153,
 0.28499653935432434,
 0.2919999957084656,
 0.3054690957069397,
 0.299551397562027,
 0.3173198103904724,
 0.3298851251602173,
 0.397148996591568,
 0.3080402612686157,
 0.36344778537750244,
 0.37432265281677246,
 0.2919999957084656]

In [26]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 2.9900452494621277
avg mae: 0.3450691948334376
avg are: 0.1489912728468577


In [27]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.8829924464225769
median testing mse: 1.1816527843475342


In [28]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.2952626794576645
median testing mae: 0.301384374499321


In [29]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.1420000046491623


In [30]:
test_ARE

[0.142,
 0.142,
 0.13883333,
 0.142,
 0.142,
 0.142,
 0.143,
 0.142,
 0.142,
 0.33197618,
 0.142,
 0.142,
 0.142,
 0.142,
 0.142,
 0.14133334,
 0.142,
 0.142,
 0.157,
 0.13894445,
 0.142,
 0.142,
 0.14183335,
 0.14,
 0.14,
 0.12653969,
 0.14177777,
 0.152,
 0.1625,
 0.142]

In [6]:
sum([0.142,
 0.142,
 0.13883333,
 0.142,
 0.142,
 0.142,
 0.143,
 0.142,
 0.142,
 0.33197618,
 0.142,
 0.142,
 0.142,
 0.142,
 0.142,
 0.14133334,
 0.142,
 0.142,
 0.157,
 0.13894445,
 0.142,
 0.142,
 0.14183335,
 0.14,
 0.14,
 0.12653969,
 0.14177777,
 0.152,
 0.1625,
 0.142][:10])/10

0.160780951